In [ ]:
# Asked ChatGPT for a baseline and this is it
# The parameters are the below along with 70 landscape parameters with 
    #Index(['DateTime', 'lat', 'lon', 'wdir15', 't15', 'ws15', 'pres15', 'wdir150',
    #       't150', 'ws150', 'pres150', 'wdir250', 't250', 'ws250', 'pres250',
    #       'wdir500', 't500', 'ws500', 'pres500', 'f', 'fg', 'gust_factor'],
    #      dtype='object')

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd, numpy as np, tensorflow as tf

In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [15]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df = df.drop(['f', 'fg', 'fsdev', 'd', 'dsdev', 'longitude', 'latitude', 'X', 'Y', 'time', 'stod'], axis = 1)# + [f'Landscape_{i}' for i in range(70)], axis = 1)

df_unfolded = df.elevations.apply(pd.Series)

df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)

In [16]:
df.iloc[:, -70:] = df.iloc[:, -70:].sub(df.station_elevation, axis = 0)

In [17]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -70:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

n_components = 5

pca = PCA(n_components=n_components)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(n_components)])

df  = pd.concat([df, compressed_df], axis = 1)

In [19]:
y = df.gust_factor
X = df.drop(['gust_factor'], axis = 1)
X = X.drop(['ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'N_01', 'N_12', 'landscape_points', 'elevations', 'station_elevation']
           + [i for i in range(70)], axis = 1)

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=2048, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(units=1, activation='linear')
])

c:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.compile(optimizer='adam', loss=mean_absolute_percentage_error)
# Train the model
model.fit(X_train, y_train, epochs = 1000, batch_size = 256, validation_data = (X_val, y_val))

Epoch 1/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 11s 68ms/step - loss: 7.6040 - val_loss: 6.8074
Epoch 2/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 7.2345 - val_loss: 6.7902
Epoch 3/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - loss: 7.1862 - val_loss: 6.7160
Epoch 4/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 7.3812 - val_loss: 6.7501
Epoch 5/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 6s 110ms/step - loss: 7.4252 - val_loss: 6.6904
Epoch 6/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - loss: 7.1441 - val_loss: 6.6884
Epoch 7/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 7.1362 - val_loss: 6.7107
Epoch 8/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 5s 68ms/step - loss: 7.0317 - val_loss: 6.7029
Epoch 9/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 7.1593 - val_loss: 6.6686
Epoch 10/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 7.0867 - val_loss: 6.7063
Epoch 11/1000
52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 7.2482 - val_loss: 6.6926
Epoch 12/1000
52/52 ━━━━━━━━━━━━━━━━━━━

In [27]:
# Evaluate the model
mape = model.evaluate(X_test, y_test)
y_predict = model.predict(X_test)
print(f'Test MAPE: {mape}%')

115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.7544
115/115 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step
Test MAPE: 6.629626274108887%


In [25]:
model.save('./saved_models/linearRegression.keras')